# BIGRoC: Boosting Image Generation via a Robust Classifier

--------------------------------------------------

## Boosting deep BigGAN on ImageNet $128\times128$

This colab notebook contains the needed code to experiment with our proposed algorithm for boosting deep BigGAN on ImageNet 128x128 (in Section 5.2).

**How to use**

1.   Upload the notebook to colab
2.   Make sure that colab uses a GPU (Edit $\rightarrow$ Notebook settings $\rightarrow$ Hardware accelerator)

**Setup:**

This notebook mounts google drive and assumes that [this directory](https://drive.google.com/drive/folders/1yN6WjMmc-pi3zHylF-I1jri7I2nZpsGJ?usp=sharing) is located in your root folder in Google Drive. Therefore, please open the Google Drive's link and choose "add a shortcut to drive" and pick "My Drive" is the chosen location. By doing so, you are ready to go :)

If you rather not to mount your google drive, download the files and edit the relevant paths accordingly.

**Credits**

This notebook uses the following packages:

* [robustness](https://github.com/MadryLab/robustness) - For adversarially robust classifier
* [guided-diffuision](https://github.com/openai/guided-diffusion/tree/main/evaluations) - ImageNet quantitative evaluation and 50K set of generated images + labels

# Setup & Installation

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install torch==1.8.2+cu102 torchvision==0.9.2+cu102 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

In [ ]:
!wget http://andrewilyas.com/ImageNet.pt

In [ ]:
!pip install robustness

In [ ]:
from robustness import model_utils, datasets
import torch


def create_dl_model(DATA='CIFAR', BATCH_SIZE=128, NUM_WORKERS=8):
    '''
    :param DATA: Choices: ['CIFAR', 'ImageNet', 'RestrictedImageNet']
    :param bs: batch size
    :param num_workers:
    :return: a dataloader object
    '''

    # Load dataset
    dataset_function = getattr(datasets, DATA)
    dataset = dataset_function('data')
    # Load model
    model_kwargs = {
        'arch': 'resnet50',
        'dataset': dataset,
        'resume_path': f'./{DATA}.pt'
    }
    model, _ = model_utils.make_and_restore_model(**model_kwargs)
    classifier = model.model
    classifier.eval()
    return classifier

adv_model = create_dl_model(DATA='ImageNet')

In [2]:
import torch
from torch import nn


class AttackerStep:
    '''
    Generic class for attacker steps, under perturbation constraints
    specified by an "origin input" and a perturbation magnitude.
    Must implement project, step, and random_perturb
    '''

    def __init__(self, orig_input, eps, step_size, use_grad=True):
        '''
        Initialize the attacker step with a given perturbation magnitude.
        Args:
            eps (float): the perturbation magnitude
            orig_input (ch.tensor): the original input
        '''
        self.orig_input = orig_input
        self.eps = eps
        self.step_size = step_size
        self.use_grad = use_grad

    def project(self, x):
        '''
        Given an input x, project it back into the feasible set
        Args:
            ch.tensor x : the input to project back into the feasible set.
        Returns:
            A `ch.tensor` that is the input projected back into
            the feasible set, that is,
        .. math:: \min_{x' \in S} \|x' - x\|_2
        '''
        raise NotImplementedError

    def step(self, x, g):
        '''
        Given a gradient, make the appropriate step according to the
        perturbation constraint (e.g. dual norm maximization for :math:`\ell_p`
        norms).
        Parameters:
            g (ch.tensor): the raw gradient
        Returns:
            The new input, a ch.tensor for the next step.
        '''
        raise NotImplementedError

    def random_perturb(self, x):
        '''
        Given a starting input, take a random step within the feasible set
        '''
        raise NotImplementedError

    def to_image(self, x):
        '''
        Given an input (which may be in an alternative parameterization),
        convert it to a valid image (this is implemented as the identity
        function by default as most of the time we use the pixel
        parameterization, but for alternative parameterizations this functino
        must be overriden).
        '''
        return x


from torch import nn

# simple Module to normalize an image
class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.mean = torch.Tensor(mean)
        self.std = torch.Tensor(std)
    def forward(self, x):
        return (x - self.mean.type_as(x)[None, :, None, None]) / self.std.type_as(x)[None, :, None, None]

norm = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])



# L2 threat model
class L2Step(AttackerStep):
    """
    Attack step for :math:`\ell_\infty` threat model. Given :math:`x_0`
    and :math:`\epsilon`, the constraint set is given by:
    .. math:: S = \{x | \|x - x_0\|_2 \leq \epsilon\}
    """

    def project(self, x):
        """
        """
        if self.orig_input is None: self.orig_input = x.detach()
        self.orig_input = self.orig_input.cuda()
        diff = x - self.orig_input
        diff = diff.renorm(p=2, dim=0, maxnorm=self.eps)
        return torch.clamp(self.orig_input + diff, 0, 1)

    def step(self, x, g):
        """
        """
        l = len(x.shape) - 1
        g_norm = torch.norm(g.reshape(g.shape[0], -1), dim=1).view(-1, *([1] * l))
        scaled_g = g / (g_norm + 1e-10)
        return x + scaled_g * self.step_size

def targeted_pgd_l2(model, X, y, num_iter, eps, step_size):
    # input images are in range [0,1]
    steper = L2Step(eps=eps, orig_input=None, step_size=step_size)
    for t in range(num_iter):
        X = X.clone().detach().requires_grad_(True).cuda()
        loss = nn.CrossEntropyLoss(reduction='none')(model(norm(X)), y)
        loss = torch.mean(loss)
        grad, = torch.autograd.grad(-1 * loss, [X])
        X = steper.step(X, grad)
        X = steper.project(X)
    return X.detach()

Donwload reference batch

In [ ]:
!wget https://openaipublic.blob.core.windows.net/diffusion/jul-2021/ref_batches/imagenet/128/VIRTUAL_imagenet128_labeled.npz

In [ ]:
!wget https://openaipublic.blob.core.windows.net/diffusion/jul-2021/ref_batches/imagenet/128/biggan_deep_trunc1_imagenet128.npz

In [3]:
import numpy as np

x = np.load("biggan_deep_trunc1_imagenet128.npz", mmap_mode='r')
gen_imgs = x['arr_0']
gt_labels = x['arr_1']

# Deep BigGAN baseline quantitative results

In [ ]:
!python ./gdrive/MyDrive/models_and_scripts/evaluator.py ./VIRTUAL_imagenet128_labeled.npz ./biggan_deep_trunc1_imagenet128.npz

downloading InceptionV3 model...
11679it [00:13, 850.48it/s]
2022-01-26 23:34:29.230569: W tensorflow/core/framework/op_def_util.cc:371] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
warming up TensorFlow...
100% 1/1 [00:08<00:00,  8.30s/it]
computing reference batch activations...
100% 157/157 [00:28<00:00,  5.53it/s]
computing/reading reference batch statistics...
computing sample batch activations...
100% 782/782 [02:15<00:00,  5.75it/s]
computing/reading sample batch statistics...
Computing evaluations...
Inception Score: 145.83079528808594
FID: 6.0220333742788625
sFID: 7.17773218489333
tcmalloc: large alloc 2000003072 bytes == 0x555b8da54000 @  0x7ff23e1e4001 0x7ff23acac54f 0x7ff23acfcb58 0x7ff23ad00b17 0x7ff23ad9f203 0x555abb18f544 0x555abb18f240 0x555abb203627 0x555abb1fdced 0x555abb190bda 0x555abb1fec0d 0x555abb190afa 0x555abb1fec0d 0x555abb190afa 0x555abb1fe915 0x555abb1fd9ee 0x555abb1fd6f3 0x55

# BIGRoC Application

In [ ]:
# calculate the debiasing factor on a subset of generated images
logits = []
with torch.no_grad():
  for idx in range(500):
    gen_imgs_debias = gen_imgs[idx * 100 : (idx + 1) * 100]
    gen_imgs_debias = torch.tensor(gen_imgs_debias / 255.).cuda().permute(0,3,1,2).float()
    logits.append(adv_model(norm(gen_imgs_debias)))
logits_t = torch.cat(logits)
debiasing = torch.ones(1).cuda() - torch.mean(logits_t, dim=0)

In [4]:
# BIGRoC args
epsilon, steps = 5, 7
step_size = 1.5 * epsilon / steps
use_pl = False

In [5]:
from tqdm.notebook import tqdm

torch.manual_seed(1234)
boosted_gen_imgs = []
for i in tqdm(range(250)):
  x_batch = gen_imgs[i * 100: (i + 1) * 100]
  x_batch_0_1 = torch.tensor(x_batch / 255.).cuda().permute(0,3,1,2).float()
  with torch.no_grad():
    if use_pl:
      labels = torch.argmax(adv_model(norm(x_batch_0_1)), dim=1)
    else:
      labels = torch.tensor(gt_labels[i * 100: (i + 1) * 100]).cuda().long()
  b_imgs = targeted_pgd_l2(model=adv_model, X=x_batch_0_1.data, y=labels.long(), num_iter=steps, eps=epsilon,
                                    step_size=step_size).detach().cpu()
  boosted_gen_imgs.append(b_imgs)

boosted_gen_imgs = torch.cat(boosted_gen_imgs)
boosted_gen_imgs = (boosted_gen_imgs * 255.).int()
boosted_gen_imgs = boosted_gen_imgs.detach().cpu().permute(0,2,3,1).numpy().astype(np.uint8)


if use_pl:
  np.savez(f"./boosted_deep_bigroc_1_eps_{epsilon}", boosted_gen_imgs)
else:
  np.savez(f"./boosted_deep_bigroc_1_eps_{epsilon}_labeled", boosted_gen_imgs)

  0%|          | 0/250 [00:00<?, ?it/s]

In [6]:
from tqdm.notebook import tqdm

torch.manual_seed(1234)
boosted_gen_imgs = []
for i in tqdm(range(250)):
  x_batch = gen_imgs[(25000 + i*100):(25000 + (i + 1) * 100)]
  x_batch_0_1 = torch.tensor(x_batch / 255.).cuda().permute(0,3,1,2).float()
  with torch.no_grad():
    if use_pl:
      labels = torch.argmax(adv_model(norm(x_batch_0_1)), dim=1)
    else:
      labels = torch.tensor(gt_labels[(25000 + i*100):(25000 + (i + 1) * 100)]).cuda().long()
  boosted_gen_imgs.append(targeted_pgd_l2(model=adv_model, X=x_batch_0_1.data, y=labels.long(), num_iter=steps, eps=epsilon,
                                    step_size=step_size).detach().cpu())

boosted_gen_imgs = torch.cat(boosted_gen_imgs)
boosted_gen_imgs = (boosted_gen_imgs * 255.).int()
boosted_gen_imgs = boosted_gen_imgs.detach().cpu().permute(0,2,3,1).numpy().astype(np.uint8)

if use_pl:
  np.savez(f"./boosted_deep_bigroc_2_eps_{epsilon}", boosted_gen_imgs)
else:
  np.savez(f"./boosted_deep_bigroc_2_eps_{epsilon}_labeled", boosted_gen_imgs)

  0%|          | 0/250 [00:00<?, ?it/s]

load boosted images

In [7]:
import numpy as np
from tqdm.notebook import tqdm
""
arr = np.zeros(shape=(50000, 128, 128, 3), dtype=np.uint8)
if use_pl:
  x = np.load(f"boosted_deep_bigroc_1_eps_{epsilon}.npz")['arr_0']
  arr[:25000] = x
  x = np.load(f"boosted_deep_bigroc_2_eps_{epsilon}.npz")['arr_0']
  arr[25000:] = x
else:
  x = np.load(f"boosted_deep_bigroc_1_eps_{epsilon}_labeled.npz")['arr_0']
  arr[:25000] = x
  x = np.load(f"boosted_deep_bigroc_2_eps_{epsilon}_labeled.npz")['arr_0']
  arr[25000:] = x

In [54]:
arr.shape, arr.min(), arr.max()

((50000, 128, 128, 3), 0, 255)

In [8]:
if use_pl:
  np.savez(f"boosted_deep_bigroc_eps_{epsilon}", arr)
else:
  np.savez(f"boosted_deep_bigroc_eps_{epsilon}_labeled", arr)

$\epsilon = 5$ BIGRoC$_{GT}$

In [15]:
!python ./gdrive/MyDrive/models_and_scripts/evaluator.py ./VIRTUAL_imagenet128_labeled.npz ./boosted_deep_bigroc_eps_5_labeled.npz

downloading InceptionV3 model...
11679it [00:16, 695.21it/s]
2022-01-27 17:22:40.350712: W tensorflow/core/framework/op_def_util.cc:371] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
warming up TensorFlow...
100% 1/1 [00:10<00:00, 10.37s/it]
computing reference batch activations...
100% 157/157 [00:52<00:00,  2.99it/s]
computing/reading reference batch statistics...
computing sample batch activations...
100% 782/782 [03:57<00:00,  3.30it/s]
computing/reading sample batch statistics...
Computing evaluations...
Inception Score: 226.1678924560547
FID: 5.708350833161319
sFID: 7.166446597951563
tcmalloc: large alloc 2000003072 bytes == 0x555704678000 @  0x7f0ed9b9b001 0x7f0ed666354f 0x7f0ed66b3b58 0x7f0ed66b7b17 0x7f0ed6756203 0x555605a44544 0x555605a44240 0x555605ab8627 0x555605ab2ced 0x555605a45bda 0x555605ab3c0d 0x555605a45afa 0x555605ab3c0d 0x555605a45afa 0x555605ab3915 0x555605ab29ee 0x555605ab26f3 0x555

$\epsilon = 5$ BIGRoC$_{PL}$

In [9]:
!python ./gdrive/MyDrive/models_and_scripts/evaluator.py ./VIRTUAL_imagenet128_labeled.npz ./boosted_deep_bigroc_eps_5.npz

2022-01-27 18:03:47.136015: W tensorflow/core/framework/op_def_util.cc:371] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
warming up TensorFlow...
100% 1/1 [00:10<00:00, 10.11s/it]
computing reference batch activations...
100% 157/157 [00:51<00:00,  3.07it/s]
computing/reading reference batch statistics...
computing sample batch activations...
100% 782/782 [03:58<00:00,  3.28it/s]
computing/reading sample batch statistics...
Computing evaluations...
Inception Score: 176.42221069335938
FID: 5.695593857996073
sFID: 7.107968374035863
tcmalloc: large alloc 2000003072 bytes == 0x55bbf1034000 @  0x7fe28da82001 0x7fe28a54a54f 0x7fe28a59ab58 0x7fe28a59eb17 0x7fe28a63d203 0x55baf340a544 0x55baf340a240 0x55baf347e627 0x55baf3478ced 0x55baf340bbda 0x55baf3479c0d 0x55baf340bafa 0x55baf3479c0d 0x55baf340bafa 0x55baf3479915 0x55baf34789ee 0x55baf34786f3 0x55baf35424c2 0x55baf354283d 0x55baf35426e6 0x55baf351a163 0x55b